In [2]:
import matplotlib as plt
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [3]:
keyWord = 'data science'
keyWord = keyWord.replace(' ', '%20')
urlorig='https://www.headhunter.com/jobs?Keywords='+keyWord+'&Radius=150&PageNumber=1&OrderBy=Relevance'
urlorig

'https://www.headhunter.com/jobs?Keywords=data%20science&Radius=150&PageNumber=1&OrderBy=Relevance'

In [56]:
def get_pages(soup_param, res_list):
    pages = soup_param.findAll("a", class_ = "searchnext")
    if len(pages) > 0:
        link = pages[0]
        link_esc = link["href"].replace(' ', '%20')
        print(link["href"])
        Jb_page = uReq("https://www.headhunter.com" + link_esc)
        Jb_page_html = Jb_page.read()
        Jb_page_soup = soup(Jb_page_html, 'html.parser')
        Jb_page.close()
        res_list.append(Jb_page_soup)
        get_pages(Jb_page_soup, res_list)


my_url = urlorig
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser')

jb_all = []


links_all_jobs = page_soup.findAll("a", class_ = "see_all_jobs_link")
for link in links_all_jobs:
    Jb_all = uReq("https://www.headhunter.com" + link["href"])
    print(link["href"])
    Jb_all_html = Jb_all.read()
    Jb_all.close()
    Jb_all_soup = soup(Jb_all_html, "html.parser")
    jb_all.append(Jb_all_soup)
    
all_pages=[]
for item in jb_all:
    get_pages(item, all_pages)

jb_all.extend(all_pages)


/jobs?CompanyFacet=%22Robert+Half+Technology%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150
/jobs?CompanyFacet=%22TEKsystems%2C+Inc%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150
/jobs?CompanyFacet=%22Robert+Half+Finance+%26+Accounting+U.S%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150


In [5]:
#single vacancy page
# my_url1 = "https://www.headhunter.com/jobs/Data-Analytics-Manager/J3M2FR6FRKHG9GZFN1W"
my_url1 = "https://www.headhunter.com/jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C"
uClient1 = uReq(my_url1)
page_html1 = uClient1.read()
uClient1.close()
page_soup1 = soup(page_html1,'html.parser')

print(page_soup1)



<!DOCTYPE html>

<html>
<head>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"13840fb14e","applicationID":"147043153","transactionName":"Jg0IFxRbWVhTEEkBBjoLCBcKG19bVBFJBgERAw8P","queueTime":0,"applicationTime":1427,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"13840fb14e",applicationID:"147043153"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof wi

In [35]:
# content = page_soup1.findAll("div", class_="content")
# print(len(content))
# for i in content:
#     print(i.text)
#     refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#     print(refID.group()[8:])

job_text = page_soup1.findAll("div", id=("description-container-premium", "description-container-standard"))
item = job_text[0]
contents = item.select("div.content")
# print(contents)

refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
if refID != None:
    print(refID.group()[8:])

descr=''
for content in contents:

    
    descr+= content.text
    
    for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        if i.text == 'Location ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Employment Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Rate ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Store Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Other Compensation: ':
            print(i.next_sibling.string.next_sibling.text)
# print(descr)

04030-0011057821
US-TX-Fort Worth
Full-Time
Year
N/A
Engineering,  Information Technology,  Design
DOE


In [57]:
titles = []
company_names = []
location_names = []
compensation = []
job_descriptions = []
job_ids = []
employment_types = []
pay_types=[]
pay_rates=[]
store_types=[]
other_compensations=[]
skills = {}
skills['Python'] = []
skills['R '] = []
skills['Matlab'] = []
skills['Excel'] = []
skills['SAS'] = []
skills['SQL'] = []
skills['SPSS'] = []
skills['Hadoop'] = []
skills['Spark'] = []

# extract job_title
jobs = page_soup.findAll("div", class_=re.compile("searchresult clearfix"))
for job in jobs:
    try:
        titles.append(job["title"])
    except:
        titles.append("NA")

# print(titles)

# extract company_name
companies = page_soup.findAll("span", class_="resultcompany_mobile clearfix" )
for company in companies:
    try:
        company_names.append(company.text.strip())
    except:
        company_names.append("NA")
        
# print(company_names)

# extract location       
locations = page_soup.findAll("span", class_="result-right-one")
for location in locations:
    try:
        location_names.append(location.text)
    except:
        location_names.append("NA")
        
# print(location_names)

# extract compensation       
payments = page_soup.findAll("span", class_="result-right-two non-mobile")
for payment in payments:
    try:
        compensation.append(payment.text)
    except:
        compensation.append("NA")
        
# print(compensation)

#extract job_id and full description
links = page_soup.findAll("a", class_ = "result-left")
ind=0
for link in links:
    ind+=1
    print(ind, link["href"])
    Jb = uReq("https://www.headhunter.com" + link["href"])
    Jb_html = Jb.read()
    Jb.close()
    Jb_soup = soup(Jb_html, "html.parser")
    
    job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
    item = job_text[0]
    contents = item.select("div.content")

    refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
    try:
        job_ids.append(refID.group()[8:])
    except:
        job_ids.append("NA")
    
    descr=''
    other_comp_flag=False
    for content in contents:
        
        descr+= content.text
    
        for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
    #         if i.text == 'Location ':
    #             print(i.next_sibling.string.next_sibling.text)

    #         other_compensations.append("NA")

            if i.text == 'Employment Type ':
                try:
                    employment_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    employment_types.append("NA")
            if i.text == 'Pay Type ':
                try:
                    pay_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_types.append("NA")
            if i.text == 'Pay Rate ':
                try:
                    pay_rates.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_rates.append("NA")
            if i.text == 'Store Type ':
                try:
                    store_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    store_types.append("NA")
            if i.text == 'Other Compensation: ':
                other_comp_flag=True
                try:
                    other_compensations.append(i.next_sibling.string.next_sibling.text)
                except:
                    other_compensations.append("NA")
                
    try:
        job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
    except:
        job_descriptions.append("NA")
    
    if other_comp_flag==False:
        other_compensations.append("NA")
    
#     try:
#         content = Jb_soup.find("div", class_="content")
#         refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#         job_ids.append(refID.group()[8:])
#     except:
#         job_ids.append("NA")    
        
#     try:
#         job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
#     except:
#         job_descriptions.append("NA")
        
        
# print(job_descriptions)

#extract all jobsfor company with many vacancies
for item in jb_all:
    
    # extract job_title
    jobs = item.findAll("div", class_=re.compile("searchresult clearfix"))
    for job in jobs:
        try:
            titles.append(job["title"])
        except:
            titles.append("NA")
            
    # extract company_name
    companies = item.findAll("span", class_="resultcompany_mobile clearfix" )
    for company in companies:
        try:
            company_names.append(company.text.strip())
        except:
            company_names.append("NA")

    # extract location       
    locations = item.findAll("span", class_="result-right-one")
    for location in locations:
        try:
            location_names.append(location.text)
        except:
            location_names.append("NA")

    # extract compensation       
    payments = item.findAll("span", class_="result-right-two non-mobile")
    for payment in payments:
        try:
            compensation.append(payment.text)
        except:
            compensation.append("NA")

    #extract job_id and full description
    links = item.findAll("a", class_ = "result-left")
    for link in links:
        ind+=1
        print(ind, link["href"])
        Jb = uReq("https://www.headhunter.com" + link["href"])
        Jb_html = Jb.read()
        Jb.close()
        Jb_soup = soup(Jb_html, "html.parser")

        job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
        item = job_text[0]
        contents = item.select("div.content")

        refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
        try:
            job_ids.append(refID.group()[8:])
        except:
            job_ids.append("NA")

        descr=''
        other_comp_flag=False
        for content in contents:
        
            descr+= content.text

            for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        #         if i.text == 'Location ':
        #             print(i.next_sibling.string.next_sibling.text)

        #         other_compensations.append("NA")

                if i.text == 'Employment Type ':
                    try:
                        employment_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        employment_types.append("NA")
                if i.text == 'Pay Type ':
                    try:
                        pay_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_types.append("NA")
                if i.text == 'Pay Rate ':
                    try:
                        pay_rates.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_rates.append("NA")
                if i.text == 'Store Type ':
                    try:
                        store_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        store_types.append("NA")
                if i.text == 'Other Compensation: ':
                    other_comp_flag=True
                    try:
                        other_compensations.append(i.next_sibling.string.next_sibling.text)
                    except:
                        other_compensations.append("NA")

        try:
            job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
        except:
            job_descriptions.append("NA")

        if other_comp_flag==False:
            other_compensations.append("NA")

1 /jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C
2 /jobs/Senior-Data-Scientist/J3M0HJ6G44D9GHZ6KSW
3 /jobs/Manager-of-IT-Audit-and-Data-Analytics/J3S11Z6T9RXPD3MPF70
4 /jobs/Senior-Data-Engineer/J3W1706KHS36HQXY3P5
5 /jobs/Data-Analytics-Manager/J3M2FR6FRKHG9GZFN1W
6 /jobs/Sr--Data-Engineer/J3W2QX77VC8F80WYZR4
7 /jobs/Associate-Director-of-Application-Engineering/J3T0PW6C5QK31S8KQX0
8 /jobs/Senior-Software-Architect/J3W11W61LDTCW604FC8
9 /jobs/Director--Analytics/J3R3L86W1CQ32H1M8FM
10 /jobs/Tax-Analyst-Manager-Director/J3W2GX73DG4MDBLBPJD
11 /jobs/Sr--Financial-Analyst-Excellent-Culture-Team--Luv-/J3V4Z869LYN28RG7T9B
12 /jobs/Sr--Financial-Analyst-Amazing-Opportunity--Luv-/J3R20C73WNC3514V9QN
13 /jobs/Sr--Biomarker-Scientist/J3N8996DSNYL9M2Q9XY
14 /jobs/Clinical-Project-Manager/J3N3126BLBKJLS89B7C
15 /jobs/Quality-Control-Manager/J3N5XQ6GVFDSTP845L2
16 /jobs/Senior-Clinical-Trial-Manager/J3V89K78457PBXV81KV
17 /jobs/Sr--Business-Analyst/J3P73X63SZHYHBYLP2L
18 /jobs/Modernization-Qual

In [58]:
print(len(other_compensations))

55


In [49]:
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 'Compensation': compensation, 
     'Job_ID':job_ids,'Job_Description':job_descriptions, 'Employment_Type':employment_types, 'Pay_Type':pay_types,
     'Pay_Rate':pay_rates, 'Store_Type':store_types, 'Other_Compensation':other_compensations}

df1 = pd.DataFrame(d)
df2 = pd.DataFrame(skills)
frames = [df1, df2]
results = pd.concat(frames, axis = 1)
results.head()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Employment_Type,Pay_Type,Pay_Rate,Store_Type,Other_Compensation,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
0,Senior-Data-Scientist,Robert Half Technology,TX - Fort Worth,N/A,04030-0011057821,Ref ID: 04030-0011057821Classification: Data S...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",DOE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Senior-Data-Scientist,Robert Half Technology,NJ - Jersey City,$140k - $160k/year,02710-0011220068,Ref ID: 02710-0011220068Classification: Data S...,Full-Time,Year,"$140,000.00 - $160,000.00 /Year","Engineering, Information Technology, Design",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Manager-of-IT-Audit-and-Data-Analytics,Robert Half Technology,GA - Atlanta,$120k - $130k/year,00900-0011225244,Ref ID: 00900-0011225244Classification: Sr. IT...,Full-Time,Year,"$120,000.00 - $130,000.00 /Year","Engineering, Information Technology, Design",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Senior-Data-Engineer,Robert Half Technology,TX - Austin,$115k - $130k/year,04160-0010956620,Ref ID: 04160-0010956620Classification: Big Da...,Full-Time,Year,"$115,000.00 - $130,000.00 /Year","Engineering, Information Technology, Design",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data-Analytics-Manager,"TEKsystems, Inc",GA - Atlanta,N/A,NA,***** CANDIDATES MUST BE ABLE TO WORK W2 ****...,Full-Time,Year,N/A,Information Technology,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
results.tail()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Employment_Type,Pay_Type,Pay_Rate,Store_Type,Other_Compensation,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
50,Sr--IT-Auditor,Robert Half Finance & Accounting U.S,NC - Charlotte,$70k - $90k/year,03190-0011206947,Ref ID: 03190-0011206947Classification: Sr. IT...,Full-Time,Year,"$70,000.00 - $90,000.00 /Year","Accounting, Finance",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Menlo-Park-Senior-Fund-Accountant,Robert Half Finance & Accounting U.S,CA - Mountain View,$85k - $105k/year,00412-9502291168,Ref ID: 00412-9502291168Classification: Menlo ...,Full-Time,Year,"$85,000.00 - $105,000.00 /Year","Accounting, Finance",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Venture-Capital-Senior-Fund-Accountant,Robert Half Finance & Accounting U.S,CA - San Francisco,$85k - $105k/year,00412-9502291173,Ref ID: 00412-9502291173Classification: Ventur...,Full-Time,Year,"$85,000.00 - $105,000.00 /Year","Accounting, Finance",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Venture-Capital-Senior-Fund-Accountant,Robert Half Finance & Accounting U.S,CA - San Jose,$85k - $105k/year,00412-9502291162,Ref ID: 00412-9502291162Classification: Ventur...,Full-Time,Year,"$85,000.00 - $105,000.00 /Year","Accounting, Finance",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Sand-Hill-Road-Senior-Fund-Accountant,Robert Half Finance & Accounting U.S,CA - San Francisco,$85k - $105k/year,00412-9502291170,Ref ID: 00412-9502291170Classification: Sand H...,Full-Time,Year,"$85,000.00 - $105,000.00 /Year","Accounting, Finance",NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
results.to_csv('Headhunters_job_vacancies_raw.csv')